In [89]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [90]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mutual_info_score
import category_encoders as ce

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import KFold
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBClassifier


In [91]:
df = pd.read_csv('../DATA/all_season_summary.csv')

In [92]:
df.head()

,season,id,name,short_name,description,home_team,away_team,toss_won,decision,1st_inning_score,2nd_inning_score,home_score,away_score,winner,result,start_date,end_date,venue_id,venue_name,home_captain,away_captain,pom,points,super_over,home_overs,home_runs,home_wickets,home_boundaries,away_overs,away_runs,away_wickets,away_boundaries,highlights,home_key_batsman,home_key_bowler,home_playx1,away_playx1,away_key_batsman,away_key_bowler,match_days,umpire1,umpire2,tv_umpire,referee,reserve_umpire
0,2023.0,1359475,Gujarat Titans v Chennai Super Kings,GT v CSK,"1st Match (N), Indian Premier League at Ahmedabad, Mar 31 2023",GT,CSK,GT,BOWL FIRST,178/7,182/5,182/5,178/7,GT,Titans won by 5 wkts (4b rem),2023-03-31T14:00Z,2023-04-01T23:59Z,57851,"Narendra Modi Stadium, Motera, Ahmedabad",Hardik Pandya,MS Dhoni,Rashid Khan,"Gujarat Titans 2, Chennai Super Kings 0",False,19.2,182.0,5.0,23.0,20.0,178.0,7.0,23.0,"Gill, Rashid lead defending champions Gujarat Titans to victory in IPL 2023 opener. Ruturaj Gaikwad scored 92 but Chennai Super Kings managed only 178 in Ahmedabad","Shubman Gill,Vijay Shankar","Rashid Khan,Mohammed Shami","Wriddhiman Saha (WK),Shubman Gill (UKN),Sai Sudharsan (UKN),Hardik Pandya (AR),Vijay Shankar (AR),Rahul Tewatia (AR),Rashid Khan (AR),Mohammed Shami (BL),Josh Little (BL),Yash Dayal (BL),Alzarri Joseph (BL),Kane Williamson (UKN)","Devon Conway (UKN),Ruturaj Gaikwad (UKN),Moeen Ali (AR),Ben Stokes (AR),Ambati Rayudu (UKN),Shivam Dube (AR),Ravindra Jadeja (AR),MS Dhoni (WK),Mitchell Santner (AR),Deepak Chahar (BL),Rajvardhan Hangargekar (AR),Tushar Deshpande (BL)","Ruturaj Gaikwad,Moeen Ali","Rajvardhan Hangargekar,Ravindra Jadeja",31 March 2023 - night match (20-over match),Saiyed Khalid,Nitin Menon,Virender Sharma,Javagal Srinath,Abhijit Bengeri
1,2023.0,1359476,Punjab Kings v Kolkata Knight Riders,PBKS v KKR,"2nd Match (D/N), Indian Premier League at Chandigarh, Apr 1 2023",PBKS,KKR,KKR,BOWL FIRST,191/5,146/7,191/5,146/7,PBKS,Punjab Kings won by 7 runs (DLS),2023-04-01T10:00Z,2023-04-02T23:59Z,57991,"Punjab Cricket Association IS Bindra Stadium, Mohali, Chandigarh",Shikhar Dhawan,Nitish Rana,Arshdeep Singh,"Punjab Kings 2, Kolkata Knight Riders 0",False,20.0,191.0,5.0,26.0,16.0,146.0,7.0,20.0,"Arshdeep, Rajapaksa lead Punjab Kings to victory in rain-hit game. Knight Riders, behind for most parts of the game, almost got to the DLS target thanks to Andre Russell and Venkatesh Iyer","Bhanuka Rajapaksa,Shikhar Dhawan","Arshdeep Singh,Rahul Chahar","Prabhsimran Singh (UKN),Shikhar Dhawan (UKN),Bhanuka Rajapaksa (UKN),Jitesh Sharma (WK),Sikandar Raza (AR),Sam Curran (AR),M Shahrukh Khan (UKN),Harpreet Brar (BL),Rahul Chahar (BL),Arshdeep Singh (BL),Nathan Ellis (BL),Rishi Dhawan (AR)","Mandeep Singh (AR),Rahmanullah Gurbaz (WK),Anukul Roy (AR),Venkatesh Iyer (AR),Nitish Rana (UKN),Rinku Singh (UKN),Andre Russell (AR),Shardul Thakur (BL),Sunil Narine (AR),Tim Southee (BL),Umesh Yadav (BL),Varun Chakravarthy (BL)","Andre Russell,Venkatesh Iyer","Tim Southee,Varun Chakravarthy",01 April 2023 - day/night match (20-over match),Yeshwant Barde,Bruce Oxenford,Jayaraman Madanagopal,Manu Nayyar,Pranav Joshi
2,2023.0,1359477,Lucknow Super Giants v Delhi Capitals,LSG v DC,"3rd Match (N), Indian Premier League at Lucknow, Apr 1 2023",LSG,DC,DC,BOWL FIRST,193/6,143/9,193/6,143/9,LSG,Super Giants won by 50 runs,2023-04-01T14:00Z,2023-04-02T23:59Z,1070094,"Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow",KL Rahul,David Warner,Mark Wood,"Lucknow Super Giants 2, Delhi Capitals 0",False,20.0,193.0,6.0,21.0,20.0,143.0,9.0,17.0,"Wood five-for, Mayers 73 stud Super Giants' win. Capitals struggled against Super Giants' bowlers in the 194 chase on a challenging pitch","Kyle Mayers,Nicholas Pooran","Mark Wood,Avesh Khan","KL Rahul (UKN),Kyle Mayers (AR),Deepak Hooda (AR),Krunal Pandya (AR),Marcus Stoinis (AR),Nicholas Pooran (WK),Ayush Badoni (UKN),Krishnappa Gowtham (AR),Avesh Khan (BL),Ravi Bishnoi (BL),Jaydev Unadkat (

In [93]:
def h2h(df):

    df = df.sort_values(by=['short_name', 'start_date'])
    df['shifted_home_win'] = df.groupby('short_name')['output'].shift(1)
    df['h2h_home_win_ratio_last_7'] = (
        df.groupby('short_name')['shifted_home_win']
        .rolling(window=7, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )

    df['shifted_runs'] = df.groupby('short_name')['home_runs'].shift(1) 
    df['h2h_avg_runs_last_7'] = (
        df.groupby('short_name')['shifted_runs']
        .rolling(window=7, min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )
    return df 

def gen_ELO(df):
    df = df.sort_values('start_date')  # Ensure time order

    # Dictionary storing current Elo ratings for each team
    current_ratings = {}

    def get_rating(team):
        # Default to 1500 if the team isn't in our dict yet
        return current_ratings.get(team, 1500)

    K = 40
    updated_rows = []
    for idx, row in df.iterrows():
        teamA = row['home_team']
        teamB = row['away_team']
        S_A = row['output']  # 1 if A wins, 0 if B wins, 0.5 tie

        R_A = get_rating(teamA)
        R_B = get_rating(teamB)

        # Expected score for Team A
        E_A = 1 / (1 + 10 ** ((R_B - R_A) / 400))

        # Update both teams
        R_A_new = R_A + K * (S_A - E_A)
        R_B_new = R_B + K * ((1 - S_A) - (1 - E_A))

        # Store the new ratings
        current_ratings[teamA] = R_A_new
        current_ratings[teamB] = R_B_new

        # Optionally add to the row’s data
        row['elo_home_before'] = R_A
        row['elo_away_before'] = R_B
        row['elo_home_after'] = R_A_new
        row['elo_away_after'] = R_B_new
        row['elo_before_diff'] = R_A - R_B
        updated_rows.append(row)

    # Create a new DataFrame with updated rating info
    df_updated = pd.DataFrame(updated_rows)
    return df_updated

def shift_and_roll_mean(df, group_col, value_col, new_col_name,
                        window=7, min_periods=1, fillna_val=0):
    """
    For the given 'value_col', do:
      1) Shift by 1 within each group.
      2) Rolling mean with the specified window.
      3) Create a new column with the result.
    """

    shifted = df.groupby(group_col)[value_col].shift(1)

    rolled = (
        shifted.groupby(df[group_col])
               .rolling(window=window, min_periods=min_periods)
               .mean()
    )
    df[new_col_name] = rolled.reset_index(level=0, drop=True)

    # Fill NaN for first matches
    df[new_col_name] = df[new_col_name].fillna(fillna_val)

    return df

def generate_new(my_df, team):
    my_df = my_df.sort_values(by=[
        team,
        'start_date'
    ])
    my_df = shift_and_roll_mean(my_df, team, 'home_runs', f'{team}_avg_runs_scored_last_7')
    my_df = shift_and_roll_mean(my_df, team, 'away_runs', f'{team}_avg_runs_conceded_last_7')
    my_df = shift_and_roll_mean(my_df, team, 'output', f'{team}_win_ratio_last_7')
    my_df = shift_and_roll_mean(my_df, team, 'run_rate', f'{team}_avg_run_rate_last_7')
    my_df = shift_and_roll_mean(my_df, team, 'bowl_econ', f'{team}_avg_bowl_econ_last_7')
    my_df = shift_and_roll_mean(my_df, team, 'home_boundaries', f'{team}_avg_boundaries_scored_last_7')
    my_df = shift_and_roll_mean(my_df, team, 'away_boundaries', f'{team}_avg_boundaries_conceded_last_7')
    return my_df

In [94]:
# Read additonal match data

In [95]:
ball_df = pd.read_csv('../DATA/all_season_details.csv')

# Group by (match_id, batsman1_id)
batsman_match_stats = (
    ball_df.groupby(['match_id', 'batsman1_id'], as_index=False)
    .agg(
        runs_scored=('runs', 'sum'),
        boundaries=('isBoundary', 'sum'),
        legal_deliveries=('comment_id', 'count')  
    )
)

# Basic strike rate if you know how many legitimate deliveries faced:
batsman_match_stats['strike_rate'] = (
    batsman_match_stats['runs_scored'] / 
    batsman_match_stats['legal_deliveries'].replace(0, pd.NA) * 100
)

bowler_match_stats = (
    ball_df.groupby(['match_id','bowler1_id'], as_index=False)
    .agg(
        runs_conceded=('runs','sum'),
        boundaries_conceded=('isBoundary','sum'),
        deliveries_bowled=('comment_id','count'),  
        wickets=('wicket_id','count')  # if wicket_id is not None on that ball
    )
)
# Economy = (runs_conceded / overs)
bowler_match_stats['overs_bowled'] = bowler_match_stats['deliveries_bowled'] / 6.0
bowler_match_stats['economy'] = (
    bowler_match_stats['runs_conceded'] /
    bowler_match_stats['overs_bowled'].replace(0, pd.NA)
)


/var/folders/zn/nr4dngtd7r9046m_s7578mjh0000gq/T/ipykernel_6000/3035150848.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  ball_df = pd.read_csv('../DATA/all_season_details.csv')


In [96]:
batsman_match_stats

,match_id,batsman1_id,runs_scored,boundaries,legal_deliveries,strike_rate
0,335982,5766,12,1,12,100.000000
1,335982,6938,11,1,12,91.666667
2,335982,7133,24,2,20,120.000000
3,335982,8291,6,0,10,60.000000
4,335982,26206,0,0,2,0.000000
...,...,...,...,...,...,...
15274,1370353,714451,32,2,21,152.380952
15275,1370353,793463,1,0,2,50.000000
15276,1370353,1060380,26,4,16,162.500000
15277,1370353,1070173,39,7,20,195.000000


In [97]:
top_scorers = (
    batsman_match_stats
    .sort_values(['match_id','runs_scored'], ascending=[True,False])
    .drop_duplicates(subset='match_id', keep='first')
    # now each match_id has the single top row
    .rename(columns={
        'batsman1_id': 'top_scorer_id',
        'runs_scored': 'top_scorer_runs',
        'strike_rate': 'top_scorer_sr'
    })
    [['match_id','top_scorer_id','top_scorer_runs','top_scorer_sr']]
)

In [98]:
top_scorers

,match_id,top_scorer_id,top_scorer_runs,top_scorer_sr
11,335982,37737,169,219.480519
18,335983,5939,117,216.666667
33,335984,28763,61,124.489796
49,335985,35582,49,128.947368
58,335986,5766,45,100.000000
...,...,...,...,...
15210,1359544,1070173,109,194.642857
15227,1370350,1060380,62,140.909091
15233,1370351,325012,46,158.620690
15261,1370352,1070173,133,211.111111


In [99]:
# df = df.merge(
#     top_scorers,
#     left_on='id',   
#     right_on='match_id',
#     how='left'
# )

# df.drop(columns=['match_id'], inplace=True)

In [100]:
bowler_match_stats

,match_id,bowler1_id,runs_conceded,boundaries_conceded,deliveries_bowled,wickets,overs_bowled,economy
0,335982,6938,41,4,25,1,4.166667,9.840000
1,335982,8291,24,4,7,0,1.166667,20.571429
2,335982,26184,25,3,28,3,4.666667,5.357143
3,335982,28779,23,2,25,3,4.166667,5.520000
4,335982,29725,26,2,18,0,3.000000,8.666667
...,...,...,...,...,...,...,...,...
12109,1370353,822553,56,10,24,0,4.000000,14.000000
12110,1370353,928067,30,3,12,0,2.000000,15.000000
12111,1370353,1138316,36,5,24,0,4.000000,9.000000
12112,1370353,1182529,17,0,21,2,3.500000,4.857143


In [101]:
top_bowlers = (
    bowler_match_stats
    .sort_values(['match_id', 'economy', 'wickets'], ascending=[True,True,False])
    .drop_duplicates(subset='match_id', keep='first')
    # now each match_id has the single top row for bowling
    .rename(columns={
        'bowler1_id': 'top_bowler_id',
        'runs_conceded': 'top_bowler_runs_conceded',
        'economy': 'top_bowler_eco',
        'wickets': 'top_bowler_wickets'
    })
    [['match_id','top_bowler_id','top_bowler_runs_conceded','top_bowler_eco', 'top_bowler_wickets']]
)
top_bowlers

,match_id,top_bowler_id,top_bowler_runs_conceded,top_bowler_eco,top_bowler_wickets
9,335982,227712,9,2.700000,2
20,335983,49636,35,8.400000,1
32,335984,49638,14,3.500000,2
43,335985,46774,17,4.250000,0
54,335986,41434,11,2.640000,1
...,...,...,...,...,...
12064,1359544,793463,24,6.000000,1
12069,1370350,234675,18,4.500000,2
12090,1370351,1206039,6,1.714286,6
12095,1370352,537119,11,4.714286,5


In [102]:
# df = df.merge(
#     top_bowlers,
#     left_on='id',    
#     right_on='match_id',
#     how='left'
# )

# df.drop(columns=['match_id'], inplace=True)

In [103]:
def diff_n_ratio(df):
    D = 1e-7  # small constant
    df['team_diff_avg_runs_scored_last_7'] = (
        df['home_team_avg_runs_scored_last_7'] 
        - df['away_team_avg_runs_scored_last_7']
    )
    df['team_diff_avg_runs_conceded_last_7'] = (
        df['home_team_avg_runs_conceded_last_7'] 
        - df['away_team_avg_runs_conceded_last_7']
    )
    df['team_ratio_avg_run_rate_last_7'] = (
        df['home_team_avg_run_rate_last_7'] + D
    ) / (
        df['away_team_avg_run_rate_last_7'] + D
    )
    df['team_ratio_win_ratio_last_7'] = (
        df['home_team_win_ratio_last_7'] + D
    ) / (
        df['away_team_win_ratio_last_7'] + D
    )
    df['team_ratio_avg_bowl_econ_last_7'] = (
        df['home_team_avg_bowl_econ_last_7'] + D
    ) / (
        df['away_team_avg_bowl_econ_last_7'] + D
    )
    df['team_ratio_avg_boundaries_scored_last_7'] = (
        df['home_team_avg_boundaries_scored_last_7'] + D
    ) / (
        df['away_team_avg_boundaries_scored_last_7'] + D
    )
    df['team_ratio_avg_boundaries_conceded_last_7'] = (
        df['home_team_avg_boundaries_conceded_last_7'] + D
    ) / (
        df['away_team_avg_boundaries_conceded_last_7'] + D
    )
    return df


In [104]:
def pre_process(df):

    mdf = df.copy()
    mdf = mdf.sort_values('start_date')  # Ensure time order

    mdf['output'] = (mdf['home_team'] == mdf['winner']).astype(int)
    mdf['toss_won'] = (mdf['home_team'] == mdf['toss_won']).astype(int)

    mdf['run_rate'] = mdf['home_runs'] / mdf['home_overs']
    mdf['bowl_econ'] = mdf['away_runs'] / mdf['away_overs']

    mdf[['season_match', 'day_night_play']] = mdf.description.str.extract(r'^(.*?)\s*\(([^)]*)\)')


    mdf = h2h(mdf)

    mdf = gen_ELO(mdf)

    inter_df = generate_new(mdf, 'home_team')
    proc_df = generate_new(inter_df, 'away_team')

    proc_df = diff_n_ratio(proc_df)

    proc_df['season_weight'] = np.exp( (proc_df['season'] - proc_df['season'].min()) / 2 )
    proc_df['run_econ_interaction'] = proc_df['home_team_avg_run_rate_last_7'] * proc_df['away_team_avg_bowl_econ_last_7']

    return proc_df


In [105]:
proc_df = pre_process(df)

In [106]:
# Cleanup


# data leakage candidates 

cols_to_remove = ['result', '1st_inning_score', '2nd_inning_score', 'home_score', 'away_score', 'home_runs', 'away_runs', 'home_wickets', 'away_wickets',
                'pom', 'points', 'highlights', 'shifted_home_win', 'shifted_runs']
proc_df.drop(columns=cols_to_remove, axis=1, inplace=True)
sec_cols_to_remove = ['id', 'description', 'name', 'venue_name', 'match_days', 'umpire1', 'umpire2', 'tv_umpire', 'referee', 'reserve_umpire']
proc_df.drop(columns=sec_cols_to_remove, axis=1, inplace=True)
ter_cols_to_remove = ['winner', 'super_over', 'home_overs', 'away_overs', 'home_boundaries', 'away_boundaries',  'run_rate', 'bowl_econ']
proc_df.drop(columns=ter_cols_to_remove, axis=1, inplace=True)

other_cols_to_remove = [
    'elo_home_after', 'elo_away_after', 'elo_home_before', 'elo_away_before',
    'home_team_avg_runs_scored_last_7', 'away_team_avg_runs_scored_last_7', 
    'home_team_avg_runs_conceded_last_7', 'away_team_avg_runs_conceded_last_7',
    'home_team_avg_run_rate_last_7', 'away_team_avg_run_rate_last_7',
    'home_team_win_ratio_last_7', 'away_team_win_ratio_last_7',
    'home_team_avg_bowl_econ_last_7', 'away_team_avg_bowl_econ_last_7',
    'home_team_avg_boundaries_scored_last_7', 'away_team_avg_boundaries_scored_last_7',
    'home_team_avg_boundaries_conceded_last_7', 'away_team_avg_boundaries_conceded_last_7'
    ]
proc_df.drop(columns=other_cols_to_remove, axis=1, inplace=True)

# other NLP fields (to be removed unless advanced analytics needed)
cat_cols_to_remove = ['short_name',  'end_date',  'home_playx1', 'away_playx1', # 'start_date',
    'away_key_batsman', 'away_key_bowler']  # 'home_captain', 'away_captain', 'home_key_batsman', 'home_key_bowler', 
proc_df.drop(columns=cat_cols_to_remove, axis=1, inplace=True)


proc_df['toss_won'] = proc_df.toss_won.fillna('')
proc_df['decision'] = proc_df.decision.fillna('')
proc_df['day_night_play'] = proc_df.day_night_play.fillna('')
proc_df['season_match'] = proc_df.season_match.fillna('')

for c in ['season', 'h2h_home_win_ratio_last_7', 'h2h_avg_runs_last_7']:
    if c in ['season']:
        max_c = proc_df[c].max()
        proc_df[c] = proc_df[c].fillna(max_c)
    else:
        mean_c = proc_df[c].mean()
        proc_df[c] = proc_df[c].fillna(mean_c)

cat_cols = [col  for col in proc_df.columns if proc_df[col].dtype == object]
for col in cat_cols:
    proc_df[col] = (
        proc_df[col]
        .str.lower()
        .str.strip()
        .str.replace(r'\s+', '_', regex=True)
)
cat_cols.extend(['venue_id',  'season', 'toss_won']) #'top_scorer_id', 'top_bowler_id',
cat_cols

['home_team',
 'away_team',
 'decision',
 'start_date',
 'home_captain',
 'away_captain',
 'home_key_batsman',
 'home_key_bowler',
 'season_match',
 'day_night_play',
 'venue_id',
 'season',
 'toss_won']

In [107]:
proc_df.head()

,season,home_team,away_team,toss_won,decision,start_date,venue_id,home_captain,away_captain,home_key_batsman,home_key_bowler,output,season_match,day_night_play,h2h_home_win_ratio_last_7,h2h_avg_runs_last_7,elo_before_diff,team_diff_avg_runs_scored_last_7,team_diff_avg_runs_conceded_last_7,team_ratio_avg_run_rate_last_7,team_ratio_win_ratio_last_7,team_ratio_avg_bowl_econ_last_7,team_ratio_avg_boundaries_scored_last_7,team_ratio_avg_boundaries_conceded_last_7,season_weight,run_econ_interaction
974,2008.0,kxip,csk,0,bat_first,2008-04-19t11:30z,57991,yuvraj_singh,ms_dhoni,"james_hopes,kumar_sangakkara","irfan_pathan,wilkin_mota",0,2nd_match,d/n,0.536194,155.083808,0.000000,0.000000,0.000000,1.000000,1.000000e+00,1.000000,1.000000,1.000000,1.0,0.000000
987,2008.0,rcb,csk,0,bat_first,2008-04-28t14:30z,57897,rahul_dravid,ms_dhoni,"ross_taylor,wasim_jaffer","zaheer_khan,praveen_kumar",0,15th_match,n,0.536194,155.083808,-75.598293,-98.500000,-60.000000,0.588428,1.000000e+00,0.798757,0.462963,0.736111,1.0,73.082781
996,2008.0,rr,csk,0,bat_first,2008-05-04t14:30z,58162,shane_warne,ms_dhoni,"graeme_smith,swapnil_asnodkar","sohail_tanvir,munaf_patel",1,24th_match,n,0.536194,155.083808,3.399723,-4.000000,-50.500000,1.012019,1.000000e+07,0.775395,1.108696,0.639344,1.0,99.702238
1000,2008.0,dc,csk,0,bowl_first,2008-05-08t10:30z,58040,virender_sehwag,ms_dhoni,"gautam_gambhir,shikhar_dhawan","pradeep_sangwan,yo_mahesh",0,28th_match,d/n,0.536194,155.083808,22.699055,0.833333,-20.666667,1.031425,2.999999e+00,0.872851,1.163793,0.810811,1.0,81.205618
1008,2008.0,mi,csk,1,bowl_first,2008-05-14t14:30z,58324,sachin_tendulkar,ms_dhoni,"sanath_jayasuriya,robin_uthappa","dhawal_kulkarni,dwayne_bravo",1,36th_match,n,0.536194,155.083808,-49.826833,-21.000000,-39.500000,0.860529,2.000000e+00,0.971799,0.891566,0.707071,1.0,69.680426


In [108]:
# Since it needs to be a time aware split, shuffle=False .. which results in past matches only for training and future matches for predicting.
proc_df = proc_df.sort_values('start_date')  # Ensure time order
# proc_df.drop(columns=['start_date'], axis=1, inplace=True)


df_full_train, df_test = train_test_split(proc_df, test_size=.2, shuffle=False, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=.25, shuffle=False, random_state=1)
print(f"Length of df_train:{len(df_train)},  df_val:{len(df_val)},  df_test:{len(df_test)}")

y_train = df_train.pop('output')
y_val = df_val.pop('output')
y_test = df_test.pop('output')
y_full_train = df_full_train.pop('output')


Length of df_train:618,  df_val:207,  df_test:207


In [109]:
df_test[df_test.season < 2020]

,season,home_team,away_team,toss_won,decision,start_date,venue_id,home_captain,away_captain,home_key_batsman,home_key_bowler,season_match,day_night_play,h2h_home_win_ratio_last_7,h2h_avg_runs_last_7,elo_before_diff,team_diff_avg_runs_scored_last_7,team_diff_avg_runs_conceded_last_7,team_ratio_avg_run_rate_last_7,team_ratio_win_ratio_last_7,team_ratio_avg_bowl_econ_last_7,team_ratio_avg_boundaries_scored_last_7,team_ratio_avg_boundaries_conceded_last_7,season_weight,run_econ_interaction


In [110]:
num_cols = list(set(df_train.columns) - set(cat_cols))
num_cols

['elo_before_diff',
 'team_diff_avg_runs_scored_last_7',
 'team_diff_avg_runs_conceded_last_7',
 'team_ratio_avg_boundaries_conceded_last_7',
 'team_ratio_win_ratio_last_7',
 'h2h_home_win_ratio_last_7',
 'run_econ_interaction',
 'team_ratio_avg_bowl_econ_last_7',
 'season_weight',
 'team_ratio_avg_run_rate_last_7',
 'team_ratio_avg_boundaries_scored_last_7',
 'h2h_avg_runs_last_7']

In [111]:
cat_cols

['home_team',
 'away_team',
 'decision',
 'start_date',
 'home_captain',
 'away_captain',
 'home_key_batsman',
 'home_key_bowler',
 'season_match',
 'day_night_play',
 'venue_id',
 'season',
 'toss_won']

In [112]:
categorical_features_small = ["day_night_play", "decision"]
high_cardinality_features = list(set(cat_cols) - set(categorical_features_small))

# Target Encoding for High Cardinality Features
target_enc = ce.TargetEncoder(cols=high_cardinality_features)
df_train_encoded = target_enc.fit_transform(df_train, y_train)
df_val_encoded = target_enc.transform(df_val)

df_train = df_train_encoded.reset_index(drop=True)
df_val = df_val_encoded.reset_index(drop=True)


# Create Pipelines for Numerical and Categorical Features

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),    # Fill missing values with mean
    ('scaler', StandardScaler())                    # Standardize numerical values
])

# Categorical Transformer for One-Hot Encoding (for low cardinality categorical features)
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Column Transformer to apply transformations to different features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, categorical_features_small)
    ],
    remainder='drop'  # Drop any columns not specified
)

In [113]:
# Logistic Regression
# for c in [0.01, .1, 1, 10, 100]:
#     model = Pipeline(steps=[
#         ('preprocessor', preprocessor),
#         ('model', LogisticRegression(random_state=1, max_iter=1000, C=c, penalty='l2'))
#     ])  

#     model.fit(df_train, y_train)
#     y_pred = model.predict(df_val)
#     print(f"C={c}, L2 penalty, Accuracy: {accuracy_score(y_val, y_pred)} , ROC: {roc_auc_score(y_val, y_pred)} ")


tscv = TimeSeriesSplit(n_splits=5)

for c in [0.01, .1, 1, 10, 100]:
    fold_aucs = []
    fold_accs = []
    for train_idx, val_idx in tscv.split(df_full_train):
        X_train_fold, X_val_fold = df_full_train.iloc[train_idx], df_full_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_full_train.iloc[train_idx], y_full_train.iloc[val_idx]

        target_enc = ce.TargetEncoder(cols=high_cardinality_features, smoothing=10)
        # Transform with target encoding and preprocessor
        X_train_enc = target_enc.fit_transform(X_train_fold, y_train_fold)
        X_val_enc   = target_enc.transform(X_val_fold)
        
        model = Pipeline([
            ('preprocessor', preprocessor),
            ('model', LogisticRegression(C=c, max_iter=1000, random_state=1, class_weight='balanced'))
        ])
        model.fit(X_train_enc, y_train_fold)

        y_pred_fold = model.predict(X_val_enc)
        fold_accs.append(accuracy_score(y_val_fold, y_pred_fold))
        fold_aucs.append(roc_auc_score(y_val_fold, y_pred_fold))

    print(f"C={c}, Accuracy={np.mean(fold_accs):.3f}, ROC AUC={np.mean(fold_aucs):.3f}")


C=0.01, Accuracy=0.515, ROC AUC=0.508
C=0.1, Accuracy=0.521, ROC AUC=0.512
C=1, Accuracy=0.531, ROC AUC=0.523
C=10, Accuracy=0.527, ROC AUC=0.521
C=100, Accuracy=0.533, ROC AUC=0.527


In [114]:
# Random Forest

for d in [3, 5, 7, 8, 10, 25, 50]:
    fold_aucs = []
    fold_accs = []
    for train_idx, val_idx in tscv.split(df_full_train):
        X_train, X_val = df_full_train.iloc[train_idx], df_full_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_full_train.iloc[train_idx], y_full_train.iloc[val_idx]

        # Transform with target encoding and preprocessor
        X_train_enc = target_enc.fit_transform(X_train, y_train_fold)
        X_val_enc   = target_enc.transform(X_val)
        
        rf_model = RandomForestClassifier(
          n_estimators=100,
          max_depth=d, 
          random_state=21,

    )
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('model',rf_model)
        ])
        pipeline.fit(X_train_enc, y_train_fold)

        y_pred_fold = pipeline.predict(X_val_enc)
        fold_accs.append(accuracy_score(y_val_fold, y_pred_fold))
        fold_aucs.append(roc_auc_score(y_val_fold, y_pred_fold))

    print(f"Depth={d}, Accuracy={np.mean(fold_accs):.3f}, ROC AUC={np.mean(fold_aucs):.3f}")


Depth=3, Accuracy=0.534, ROC AUC=0.516
Depth=5, Accuracy=0.533, ROC AUC=0.519
Depth=7, Accuracy=0.530, ROC AUC=0.522
Depth=8, Accuracy=0.524, ROC AUC=0.517
Depth=10, Accuracy=0.527, ROC AUC=0.520
Depth=25, Accuracy=0.530, ROC AUC=0.524
Depth=50, Accuracy=0.526, ROC AUC=0.520


In [115]:
for ne in [20, 25, 40, 60, 100, 200]:
    for d in [3, 5, 7, 10, 12, 15]:

        fold_aucs = []
        fold_accs = []
        for train_idx, val_idx in tscv.split(df_full_train):
            X_train, X_val = df_full_train.iloc[train_idx], df_full_train.iloc[val_idx]
            y_train_fold, y_val_fold = y_full_train.iloc[train_idx], y_full_train.iloc[val_idx]

            # Transform with target encoding and preprocessor
            X_train_enc = target_enc.fit_transform(X_train, y_train_fold)
            X_val_enc   = target_enc.transform(X_val)
            
            rf_model = RandomForestClassifier(
            n_estimators=ne,
            max_depth=d, 
            random_state=21,

        )
            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('model',rf_model)
            ])
            pipeline.fit(X_train_enc, y_train_fold)

            y_pred_fold = pipeline.predict(X_val_enc)
            fold_accs.append(accuracy_score(y_val_fold, y_pred_fold))
            fold_aucs.append(roc_auc_score(y_val_fold, y_pred_fold))

        print(f"NE={ne} | Depth={d} => Accuracy={np.mean(fold_accs):.3f}, ROC AUC={np.mean(fold_aucs):.3f}")
    print()

NE=20 | Depth=3 => Accuracy=0.524, ROC AUC=0.506
NE=20 | Depth=5 => Accuracy=0.537, ROC AUC=0.526
NE=20 | Depth=7 => Accuracy=0.523, ROC AUC=0.514
NE=20 | Depth=10 => Accuracy=0.531, ROC AUC=0.525
NE=20 | Depth=12 => Accuracy=0.518, ROC AUC=0.512
NE=20 | Depth=15 => Accuracy=0.502, ROC AUC=0.498

NE=25 | Depth=3 => Accuracy=0.527, ROC AUC=0.509
NE=25 | Depth=5 => Accuracy=0.524, ROC AUC=0.512
NE=25 | Depth=7 => Accuracy=0.543, ROC AUC=0.533
NE=25 | Depth=10 => Accuracy=0.546, ROC AUC=0.539
NE=25 | Depth=12 => Accuracy=0.524, ROC AUC=0.517
NE=25 | Depth=15 => Accuracy=0.505, ROC AUC=0.498

NE=40 | Depth=3 => Accuracy=0.523, ROC AUC=0.507
NE=40 | Depth=5 => Accuracy=0.524, ROC AUC=0.512
NE=40 | Depth=7 => Accuracy=0.530, ROC AUC=0.521
NE=40 | Depth=10 => Accuracy=0.542, ROC AUC=0.533
NE=40 | Depth=12 => Accuracy=0.533, ROC AUC=0.526
NE=40 | Depth=15 => Accuracy=0.530, ROC AUC=0.526

NE=60 | Depth=3 => Accuracy=0.534, ROC AUC=0.516
NE=60 | Depth=5 => Accuracy=0.527, ROC AUC=0.513
NE=60 | 

In [116]:
for ne in [20, 25, 50, 100 ]:
    for d in [3, 8, 9, 10]:

        fold_aucs = []
        fold_accs = []
        for train_idx, val_idx in tscv.split(df_full_train):
            X_train, X_val = df_full_train.iloc[train_idx], df_full_train.iloc[val_idx]
            y_train_fold, y_val_fold = y_full_train.iloc[train_idx], y_full_train.iloc[val_idx]

            # Transform with target encoding and preprocessor
            X_train_enc = target_enc.fit_transform(X_train, y_train_fold)
            X_val_enc   = target_enc.transform(X_val)
            
            rf_model = RandomForestClassifier(
            n_estimators=ne,
            max_depth=d, 
            random_state=21,

        )
            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('model',rf_model)
            ])
            pipeline.fit(X_train_enc, y_train_fold)

            y_pred_fold = pipeline.predict(X_val_enc)
            fold_accs.append(accuracy_score(y_val_fold, y_pred_fold))
            fold_aucs.append(roc_auc_score(y_val_fold, y_pred_fold))

        print(f"NE={ne} | Depth={d} => Accuracy={np.mean(fold_accs):.3f}, ROC AUC={np.mean(fold_aucs):.3f}")
    print()

NE=20 | Depth=3 => Accuracy=0.524, ROC AUC=0.506
NE=20 | Depth=8 => Accuracy=0.531, ROC AUC=0.526
NE=20 | Depth=9 => Accuracy=0.537, ROC AUC=0.529
NE=20 | Depth=10 => Accuracy=0.531, ROC AUC=0.525

NE=25 | Depth=3 => Accuracy=0.527, ROC AUC=0.509
NE=25 | Depth=8 => Accuracy=0.530, ROC AUC=0.524
NE=25 | Depth=9 => Accuracy=0.536, ROC AUC=0.527
NE=25 | Depth=10 => Accuracy=0.546, ROC AUC=0.539

NE=50 | Depth=3 => Accuracy=0.539, ROC AUC=0.521
NE=50 | Depth=8 => Accuracy=0.524, ROC AUC=0.516
NE=50 | Depth=9 => Accuracy=0.531, ROC AUC=0.521
NE=50 | Depth=10 => Accuracy=0.523, ROC AUC=0.515

NE=100 | Depth=3 => Accuracy=0.534, ROC AUC=0.516
NE=100 | Depth=8 => Accuracy=0.524, ROC AUC=0.517
NE=100 | Depth=9 => Accuracy=0.523, ROC AUC=0.515
NE=100 | Depth=10 => Accuracy=0.527, ROC AUC=0.520



In [117]:


X_full = df_full_train.reset_index(drop=True)
y_full = y_full_train.reset_index(drop=True)





In [118]:

for ne in [20, 25,30, 40, 45, 50, 100,  500]:

    fold_aucs = []
    fold_accs = []
    for train_idx, val_idx in tscv.split(df_full_train):
        X_train, X_val = df_full_train.iloc[train_idx], df_full_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_full_train.iloc[train_idx], y_full_train.iloc[val_idx]

        # Transform with target encoding and preprocessor
        X_train_enc = target_enc.fit_transform(X_train, y_train_fold)
        X_val_enc   = target_enc.transform(X_val)
        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),  
            ('model', XGBClassifier(
                n_estimators=ne,
                learning_rate=0.01,
                random_state=42,
                eval_metric='logloss'
            ))
        ])
        pipeline.fit(X_train_enc, y_train_fold)

        y_pred_fold = pipeline.predict(X_val_enc)
        fold_accs.append(accuracy_score(y_val_fold, y_pred_fold))
        fold_aucs.append(roc_auc_score(y_val_fold, y_pred_fold))

    print(f"NE={ne}, Accuracy={np.mean(fold_accs):.3f}, ROC AUC={np.mean(fold_aucs):.3f}")



NE=20, Accuracy=0.511, ROC AUC=0.485
NE=25, Accuracy=0.514, ROC AUC=0.492
NE=30, Accuracy=0.511, ROC AUC=0.491
NE=40, Accuracy=0.508, ROC AUC=0.491
NE=45, Accuracy=0.499, ROC AUC=0.484
NE=50, Accuracy=0.498, ROC AUC=0.484
NE=100, Accuracy=0.496, ROC AUC=0.487
NE=500, Accuracy=0.514, ROC AUC=0.509


In [119]:
n_estimators = 25

In [120]:
for lr in [.001, .005, .01, .03, .05, .1,  1]: 
    fold_aucs = []
    fold_accs = []
    for train_idx, val_idx in tscv.split(df_full_train):
        X_train, X_val = df_full_train.iloc[train_idx], df_full_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_full_train.iloc[train_idx], y_full_train.iloc[val_idx]

        # Transform with target encoding and preprocessor
        X_train_enc = target_enc.fit_transform(X_train, y_train_fold)
        X_val_enc   = target_enc.transform(X_val)
        
        pipeline = Pipeline([
            ('preprocessor', preprocessor),  
            ('model', XGBClassifier(
                n_estimators=n_estimators,
                learning_rate=lr,
                random_state=42,
                eval_metric='logloss'
            ))
        ])
        pipeline.fit(X_train_enc, y_train_fold)

        y_pred_fold = pipeline.predict(X_val_enc)
        fold_accs.append(accuracy_score(y_val_fold, y_pred_fold))
        fold_aucs.append(roc_auc_score(y_val_fold, y_pred_fold))

    print(f"LR={lr}, Accuracy={np.mean(fold_accs):.3f}, ROC AUC={np.mean(fold_aucs):.3f}")



LR=0.001, Accuracy=0.537, ROC AUC=0.500
LR=0.005, Accuracy=0.526, ROC AUC=0.494
LR=0.01, Accuracy=0.514, ROC AUC=0.492
LR=0.03, Accuracy=0.518, ROC AUC=0.507
LR=0.05, Accuracy=0.505, ROC AUC=0.496
LR=0.1, Accuracy=0.498, ROC AUC=0.492
LR=1, Accuracy=0.518, ROC AUC=0.516


In [121]:
learning_rate = 0.03

In [122]:
for ne in [50, 75, 100]:
    for lr in [.01, 0.03]:
        for md in [3, 5, 7, 20]:
            for subs in [.6, .7]:
                for cs in [.7, .8]:
                    model = XGBClassifier(
                        n_estimators=ne,
                        learning_rate=lr,
                        max_depth=md,
                        subsample=subs,
                        colsample_bytree=cs,
                        random_state=42,
                        eval_metric='logloss'
                    )

                    fold_aucs = []
                    fold_accs = []
                    for train_idx, val_idx in tscv.split(df_full_train):
                        X_train, X_val = df_full_train.iloc[train_idx], df_full_train.iloc[val_idx]
                        y_train_fold, y_val_fold = y_full_train.iloc[train_idx], y_full_train.iloc[val_idx]

                        # Transform with target encoding and preprocessor
                        X_train_enc = target_enc.fit_transform(X_train, y_train_fold)
                        X_val_enc   = target_enc.transform(X_val)
                        
                        pipeline = Pipeline([
                            ('preprocessor', preprocessor),  
                            ('model', model),
                        ])
                        pipeline.fit(X_train_enc, y_train_fold)

                        y_pred_fold = pipeline.predict(X_val_enc)
                        fold_accs.append(accuracy_score(y_val_fold, y_pred_fold))
                        fold_aucs.append(roc_auc_score(y_val_fold, y_pred_fold))

                    print(f"NE={ne}|LR={lr}|MD={md}|Subs={subs}|CS={cs} Accuracy={np.mean(fold_accs):.3f}, ROC AUC={np.mean(fold_aucs):.3f}")
                print("-" * 25)
            print("=" * 25)
        print()
    print("+" * 25)
                    

NE=50|LR=0.01|MD=3|Subs=0.6|CS=0.7 Accuracy=0.545, ROC AUC=0.517
NE=50|LR=0.01|MD=3|Subs=0.6|CS=0.8 Accuracy=0.547, ROC AUC=0.520
-------------------------
NE=50|LR=0.01|MD=3|Subs=0.7|CS=0.7 Accuracy=0.545, ROC AUC=0.516
NE=50|LR=0.01|MD=3|Subs=0.7|CS=0.8 Accuracy=0.547, ROC AUC=0.519
-------------------------
NE=50|LR=0.01|MD=5|Subs=0.6|CS=0.7 Accuracy=0.549, ROC AUC=0.525
NE=50|LR=0.01|MD=5|Subs=0.6|CS=0.8 Accuracy=0.546, ROC AUC=0.524
-------------------------
NE=50|LR=0.01|MD=5|Subs=0.7|CS=0.7 Accuracy=0.546, ROC AUC=0.519
NE=50|LR=0.01|MD=5|Subs=0.7|CS=0.8 Accuracy=0.562, ROC AUC=0.538
-------------------------
NE=50|LR=0.01|MD=7|Subs=0.6|CS=0.7 Accuracy=0.539, ROC AUC=0.518
NE=50|LR=0.01|MD=7|Subs=0.6|CS=0.8 Accuracy=0.545, ROC AUC=0.523
-------------------------
NE=50|LR=0.01|MD=7|Subs=0.7|CS=0.7 Accuracy=0.546, ROC AUC=0.521
NE=50|LR=0.01|MD=7|Subs=0.7|CS=0.8 Accuracy=0.569, ROC AUC=0.546
-------------------------
NE=50|LR=0.01|MD=20|Subs=0.6|CS=0.7 Accuracy=0.542, ROC AUC=0.52

In [123]:
# Final model that has reasonable acc/auc for T20 IPL data.
fold_aucs = []
fold_accs = []

model = XGBClassifier(
            n_estimators=50,
            learning_rate=0.01,
            max_depth=20,
            subsample=0.7,
            colsample_bytree=0.8,
            random_state=42,
            eval_metric='logloss'
        )


for train_idx, val_idx in tscv.split(df_full_train):
    X_train, X_val = df_full_train.iloc[train_idx], df_full_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_full_train.iloc[train_idx], y_full_train.iloc[val_idx]

    # Transform with target encoding and preprocessor
    X_train_enc = target_enc.fit_transform(X_train, y_train_fold)
    X_val_enc   = target_enc.transform(X_val)

    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),  
        ('model', model)
    ])
    pipeline.fit(X_train_enc, y_train_fold)

    y_pred_fold = pipeline.predict(X_val_enc)
    fold_accs.append(accuracy_score(y_val_fold, y_pred_fold))
    fold_aucs.append(roc_auc_score(y_val_fold, y_pred_fold))

print(f"Accuracy={np.mean(fold_accs):.3f}, ROC AUC={np.mean(fold_aucs):.3f}")

Accuracy=0.559, ROC AUC=0.537


In [124]:

# Test
X_test   = target_enc.transform(df_test)

y_pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
print(f"Accuracy={acc:.3f}, ROC AUC={auc:.3f}")

Accuracy=0.488, ROC AUC=0.506


In [125]:
df_test.head()

,season,home_team,away_team,toss_won,decision,start_date,venue_id,home_captain,away_captain,home_key_batsman,home_key_bowler,season_match,day_night_play,h2h_home_win_ratio_last_7,h2h_avg_runs_last_7,elo_before_diff,team_diff_avg_runs_scored_last_7,team_diff_avg_runs_conceded_last_7,team_ratio_avg_run_rate_last_7,team_ratio_win_ratio_last_7,team_ratio_avg_bowl_econ_last_7,team_ratio_avg_boundaries_scored_last_7,team_ratio_avg_boundaries_conceded_last_7,season_weight,run_econ_interaction
206,2021.0,csk,dc,0,bowl_first,2021-04-10t14:00z,58324,ms_dhoni,rishabh_pant,"suresh_raina,moeen_ali","shardul_thakur,dwayne_bravo",2nd_match,n,0.857143,166.714286,7.662288,-23.000000,-10.285714,8.575404e-01,5.000001e-01,1.012942e+00,7.535211e-01,9.703704e-01,665.141633,59.077438
205,2021.0,srh,kkr,1,bowl_first,2021-04-11t14:00z,58008,david_warner,eoin_morgan,"manish_pandey,jonny_bairstow","rashid_khan,mohammad_nabi",3rd_match,n,0.571429,164.428571,78.152490,-2.714286,-10.714286,9.836908e-01,8.000000e-01,9.685689e-01,9.395973e-01,9.071429e-01,665.141633,71.282387
204,2021.0,rr,pbks,1,bowl_first,2021-04-12t14:00z,58324,sanju_samson,kl_rahul,"sanju_samson,jos_buttler","chetan_sakariya,chris_morris",4th_match,n,0.536194,155.083808,-0.137865,177.714286,187.285714,9.113211e+07,4.285715e+06,9.520798e+07,2.142857e+08,2.285714e+08,665.141633,0.000000
203,2021.0,kkr,mi,1,bowl_first,2021-04-13t14:00z,58008,eoin_morgan,rohit_sharma,"nitish_rana,shubman_gill","andre_russell,pat_cummins",5th_match,n,0.428571,164.142857,-118.972493,9.000000,-17.714286,1.032248e+00,2.000000e+00,8.385403e-01,1.191667e+00,7.682119e-01,665.141633,69.769972
202,2021.0,srh,rcb,1,bowl_first,2021-04-14t14:00z,58008,david_warner,virat_kohli,"david_warner,manish_pandey","jason_holder,rashid_khan",6th_match,n,0.714286,175.285714,62.474775,19.714286,10.571429,1.135001e+00,1.333333e+00,1.041897e+00,1.092308e+00,1.236364e+00,665.141633,68.769315


In [126]:
df_full_train[df_full_train.season > 2015]

,season,home_team,away_team,toss_won,decision,start_date,venue_id,home_captain,away_captain,home_key_batsman,home_key_bowler,season_match,day_night_play,h2h_home_win_ratio_last_7,h2h_avg_runs_last_7,elo_before_diff,team_diff_avg_runs_scored_last_7,team_diff_avg_runs_conceded_last_7,team_ratio_avg_run_rate_last_7,team_ratio_win_ratio_last_7,team_ratio_avg_bowl_econ_last_7,team_ratio_avg_boundaries_scored_last_7,team_ratio_avg_boundaries_conceded_last_7,season_weight,run_econ_interaction
448,2016.0,mi,rps,1,bat_first,2016-04-09t14:30z,58324,rohit_sharma,ms_dhoni,"harbhajan_singh,ambati_rayudu","harbhajan_singh,shreyas_gopal",1st_match,n,0.536194,155.083808,143.916798,172.285714,176.428571,8.653923e+07,5.714287e+06,9.117770e+07,2.200000e+08,2.414286e+08,54.598150,0.000000
449,2016.0,kkr,dc,1,bowl_first,2016-04-10t14:30z,57980,gautam_gambhir,zaheer_khan,"gautam_gambhir,robin_uthappa","brad_hogg,andre_russell",2nd_match,n,0.571429,151.166667,205.350293,31.142857,-1.119048,1.200473e+00,1.000000e+00,9.989879e-01,1.381579e+00,1.060606e+00,54.598150,73.471175
450,2016.0,kxip,gl,0,bowl_first,2016-04-11t14:30z,57991,david_miller,suresh_raina,"murali_vijay,manan_vohra","sandeep_sharma,marcus_stoinis",3rd_match,n,0.536194,155.083808,-55.979188,138.714286,147.142857,7.692857e+07,1.428572e+06,8.353371e+07,1.442857e+08,1.828571e+08,54.598150,0.000000
451,2016.0,rcb,srh,0,bowl_first,2016-04-12t14:30z,57897,virat_kohli,david_warner,"ab_de_villiers,virat_kohli","shane_watson,yuzvendra_chahal",4th_match,n,0.666667,162.000000,19.650815,-18.000000,-1.857143,1.032125e+00,1.000000e+00,1.149724e+00,9.733333e-01,1.184755e+00,54.598150,71.529547
452,2016.0,kkr,mi,0,bowl_first,2016-04-13t14:30z,57980,gautam_gambhir,rohit_sharma,"gautam_gambhir,manish_pandey","piyush_chawla,kuldeep_yadav",5th_match,n,0.428571,153.571429,-33.218282,0.595238,-18.357143,1.048149e+00,4.999997e+00,8.712234e-01,1.122807e+00,8.565401e-01,54.598150,76.160192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,2020.0,dc,mi,1,bowl_first,2020-11-05t14:00z,392627,shreyas_iyer,rohit_sharma,"marcus_stoinis,axar_patel","ravichandran_ashwin,marcus_stoinis",qualifier_1,n,0.571429,136.857143,-95.103387,11.571429,-5.571429,1.044174e+00,2.000000e+00,9.114178e-01,1.015504e+00,8.843537e-01,403.428793,75.271514
265,2020.0,rcb,srh,0,bowl_first,2020-11-06t14:00z,59396,virat_kohli,david_warner,"ab_de_villiers,aaron_finch","mohammed_siraj,adam_zampa",eliminator,n,0.428571,184.833333,-114.351198,9.857143,11.142857,1.056223e+00,6.666667e-01,1.065049e+00,1.043478e+00,1.094017e+00,403.428793,63.781148
266,2020.0,dc,srh,1,bat_first,2020-11-08t14:00z,59396,shreyas_iyer,david_warner,"shikhar_dhawan,shimron_hetmyer","kagiso_rabada,marcus_stoinis",qualifier_2,n,0.428571,162.142857,-66.342393,12.142857,19.857143,1.093815e+00,2.000000e+00,1.100569e+00,1.212963e+00,1.168067e+00,403.428793,63.853336
267,2020.0,dc,mi,1,bat_first,2020-11-10t14:00z,392627,shreyas_iyer,rohit_sharma,"shreyas_iyer,rishabh_pant","anrich_nortje,marcus_stoinis",final,n,0.428571,139.285714,-100.646489,16.000000,1.285714,1.072399e+00,2.499999e+00,9.491900e-01,1.070312e+00,9.539474e-01,403.428793,77.398264
